In [1]:
import numpy as np
import numpy.linalg as la

### Generate random rank-2 matrix

In [2]:
A = np.array ([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]).T

np.random.seed(0)
B = np.floor(np.random.rand (2, 18) * 10) 
X = A @ B 
print(f'X = \n {X}')
print("Shape of X", X.shape)
n, m = X.shape
print(f'Rank X = {la.matrix_rank(X)}') 
U, S, V_T = la.svd(X , full_matrices=False)
print(f'S = \n {S}')

X = 
 [[12. 15. 15. 12.  8. 13.  5. 14. 10. 12. 12.  9.  7. 16.  4.  5.  0. 14.]
 [17. 22. 21. 17. 12. 19.  9. 22. 19. 15. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29. 27. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [32. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0. 46.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32. 61.  4.  5.  0. 54.]
 [42. 57. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  4.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63. 52. 40. 61. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78. 69. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  5.  0. 86.]]
Shape of X (10, 18)
Rank X = 2
S = 
 [5.40630990e+02 1.74106925e+01 5.63028215e-14 1.11208018e-14
 7.19655392e-15 5.05664240e-15 3.25453003e-15 2.47025737e-15
 7.07921719e-16 3.19489295e-16]


In [3]:
fractionObserved = .9 
np.random.seed(0)
Omega = np.array(np.random.rand(X.shape [0],X.shape [1]) < fractionObserved)
Xobs = Omega * X
# Where matrix = 0: we are missing
print(f'Observed X = \n {Xobs}')

Observed X = 
 [[12. 15. 15. 12.  8. 13.  5. 14.  0. 12. 12.  9.  7.  0.  4.  5.  0. 14.]
 [17. 22.  0. 17. 12. 19.  9. 22. 19.  0. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29.  0. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [ 0. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0.  0.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32.  0.  4.  5.  0. 54.]
 [42.  0. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  0.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63.  0. 40.  0. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78.  0. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  0.  0. 86.]]


In [4]:
Omega

array([[ True,  True,  True,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True, False,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
        False,  True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True],
       [False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False,  True,  True,  True,  True],
       [ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True, False,  True,  True,  True],

In [45]:

def shrinkage(X, tau):
    '''
    Return matrix using only singular values greater than tau
    '''
    U, S, V_T = la.svd(X, full_matrices=False)
    # shrink S
    S_star = np.where(S - tau < 0, 0, S - tau)
    S_hat = np.diag(S_star)
    return U @ S_hat @ V_T

X_star = shrinkage(X, tau=1)
X_star


array([[11.87, 14.89, 14.81, 11.87,  7.96, 12.9 ,  5.1 , 14.02, 10.27,
        11.71, 12.02,  9.  ,  7.09, 16.01,  3.82,  4.78,  0.  , 14.02],
       [16.88, 21.89, 20.83, 16.88, 11.96, 18.9 ,  9.07, 21.99, 19.2 ,
        14.75, 19.  , 13.99, 12.06, 24.98,  3.85,  4.82,  0.  , 21.99],
       [21.89, 28.89, 26.85, 21.89, 15.96, 24.91, 13.04, 29.97, 28.14,
        17.79, 25.98, 18.98, 17.03, 33.96,  3.88,  4.85,  0.  , 29.97],
       [26.9 , 35.9 , 32.86, 26.9 , 19.95, 30.91, 17.02, 37.95, 37.07,
        20.83, 32.96, 23.96, 22.01, 42.94,  3.91,  4.89,  0.  , 37.95],
       [31.91, 42.9 , 38.88, 31.91, 23.95, 36.91, 20.99, 45.93, 46.  ,
        23.87, 39.94, 28.95, 26.98, 51.92,  3.94,  4.93,  0.  , 45.93],
       [36.92, 49.9 , 44.9 , 36.92, 27.95, 42.91, 24.97, 53.91, 54.94,
        26.91, 46.92, 33.94, 31.95, 60.89,  3.97,  4.96,  0.  , 53.91],
       [41.93, 56.9 , 50.91, 41.93, 31.94, 48.91, 28.94, 61.88, 63.87,
        29.95, 53.9 , 38.93, 36.93, 69.87,  4.  ,  5.  ,  0.  , 61.88],

In [46]:
def ADMM(A_l, B_l, X, epsilon, beta, max_iter_inner):
    # Initialize Alg. 2
    X_k = X
    W_k = X_k
    Y_k = X_k

    for k in range(max_iter_inner):
        #beta += 1
        X_k_1 = shrinkage(W_k - ((1/beta) * Y_k), tau=(1/beta))

        W_k_1 = X_k_1 + (1/beta) * (A_l.T@B_l + Y_k)

        W_k_1 = (1 - Omega) * W_k_1 + Xobs 

        Y_k_1 = Y_k + beta * (X_k_1 - W_k_1)

        FN_inner = la.norm(X_k_1 - X_k, ord='fro')
        if FN_inner < epsilon:
            print("BROKE")
            break

        # Update X, Y, W for next iteration
        X_k, Y_k, W_k = X_k_1, Y_k_1, W_k_1

    print("Final FN_inner", FN_inner)
    return X_k_1

In [50]:
# Truncated NNM

# initialize with Xobs
# Set epsilon
eps_outer = .001
eps_inner = .001
beta = 1

# Set max iterations
max_iter_outer = 1000
max_iter_inner = 1000

# Set rank as 2
r = 2
# Initialize with Xobs
orig_X = Xobs

# Step 1
for l in range(max_iter_outer):
    # Take SVD of X_observed
    U, S, V_T = la.svd(orig_X , full_matrices=True)
    V = V_T.T
    # Get truncated U and V as A and B
    A_l = U[:, :r].T
    B_l = V[:, :r].T

    new_X = ADMM(A_l, B_l, orig_X, eps_inner, beta, max_iter_inner)
    #print(f"new X_l after minimization = \n {X_l_1}")


    # Only keep the new values
    #X_l_1 = Xobs * Omega + X_l_1 * (1-Omega)

    FN_outer = la.norm(new_X - orig_X, ord='fro')
    print("FN_outer", FN_outer)
    if FN_outer < eps_outer:
        break
    
    # Else, update X
    orig_X = new_X

np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)
print(f"FINAL X = \n {new_X}")






BROKE
Final FN_inner 0.0009924409281092219
FN_outer 151.94716025843158
BROKE
Final FN_inner 0.0006102164863315866
FN_outer 0.002393662878553796
BROKE
Final FN_inner 0.0006113540237218804
FN_outer 6.609045336299033e-06
FINAL X = 
 [[12. 15. 15. 12.  8. 13.  5. 14. 10. 12. 12.  9.  7. 16.  4.  5.  0. 14.]
 [17. 22. 21. 17. 12. 19.  9. 22. 19. 15. 19. 14. 12. 25.  4.  5.  0. 22.]
 [22. 29. 27. 22. 16. 25. 13. 30. 28. 18. 26. 19. 17. 34.  4.  5.  0. 30.]
 [27. 36. 33. 27. 20. 31. 17. 38. 37. 21. 33. 24. 22. 43.  4.  5.  0. 38.]
 [32. 43. 39. 32. 24. 37. 21. 46. 46. 24. 40. 29. 27. 52.  4.  5.  0. 46.]
 [37. 50. 45. 37. 28. 43. 25. 54. 55. 27. 47. 34. 32. 61.  4.  5.  0. 54.]
 [42. 57. 51. 42. 32. 49. 29. 62. 64. 30. 54. 39. 37. 70.  4.  5.  0. 62.]
 [47. 64. 57. 47. 36. 55. 33. 70. 73. 33. 61. 44. 42. 79.  4.  5.  0. 70.]
 [52. 71. 63. 52. 40. 61. 37. 78. 82. 36. 68. 49. 47. 88.  4.  5.  0. 78.]
 [57. 78. 69. 57. 44. 67. 41. 86. 91. 39. 75. 54. 52. 97.  4.  5.  0. 86.]]
